### Model API Workflow

1. CloudWatch Events로부터 호출 받을 때, 시간을 보내거나, image 내에서 현재 시간 얻어내거나. (난이도로 봤을 때 훨씬 쉬운건 아무것도 안보내고, image 내에서 진행하는 것.)

2. 시간을 20분 단위로 내림하기.

3. RDS로부터 내림한 시간을 기준으로 데이터 받음. (evs info & timestamp's sequence(length: n) & pymysql)

4. 받은 데이터 utils.py 돌리고 model 실행.

5. RDS로부터 데이터 전달하여 update.

### 내림 시간 계산

In [154]:
import datetime

def get_dt():
    now = datetime.datetime.now()
    dt = datetime.datetime(now.year, now.month, now.day, now.hour, now.minute//20*20, 00)
    str_dt = dt.strftime('%Y-%m-%d %H:%M:%S')
    return dt, str_dt

get_dt()

(datetime.datetime(2023, 8, 14, 16, 40), '2023-08-14 16:40:00')

### DB 전, 후 Datetime 계산 (sequence)

In [169]:
def get_in_dt(n):
    in_dt = []
    dt, _ = get_dt()
    for i in range(n):
        if i == 0 or i == n-1:
            idt = dt - datetime.timedelta(minutes = 20*(i+1))
            in_dt.append(idt.strftime('%Y-%m-%d %H:%M:%S'))
    return in_dt

get_in_dt(12)

['2023-08-14 16:20:00', '2023-08-14 12:40:00']

In [170]:
def get_out_dt(n):
    out_dt = []
    dt, _ = get_dt()
    for i in range(n):
        if i == 0 or i == n-1:
            odt = dt + datetime.timedelta(minutes = 20*(i+1))
            out_dt.append(odt.strftime('%Y-%m-%d %H:%M:%S'))
    return out_dt

get_out_dt(6)

['2023-08-14 17:00:00', '2023-08-14 18:40:00']

### Insert Sequence Values Into Table

In [8]:
import pandas as pd
occupancy = pd.read_csv('../data/inference_data_220320sun_200326satur.csv')

In [9]:
# 차후 seq 데이터 변형하기 위한 코드.
occupancy = occupancy.set_index('time').stack().reset_index()
occupancy.columns = ['time', 'sid', 'occupancy']
occupancy

,time,sid,occupancy
0,2022-03-20 00:00:00,강원_강릉_HAPPY700 평창시네마,0.0
1,2022-03-20 00:00:00,강원_강릉_감자연구소,0.0
2,2022-03-20 00:00:00,강원_강릉_강남동주민센터,0.0
3,2022-03-20 00:00:00,강원_강릉_강릉축협 하나로마트,0.0
4,2022-03-20 00:00:00,강원_강릉_강원도교육연수원,0.0
...,...,...,...
475267,2022-03-26 23:40:00,충북_충주_중앙탑면 행정복지센터,0.0
475268,2022-03-26 23:40:00,충북_충주_충주시청 민원주차장,0.0
475269,2022-03-26 23:40:00,충북_충주_충주실내체육관,0.0
475270,2022-03-26 23:40:00,충북_충주_충주종합스포츠타운,0.0


In [60]:
attrs = pd.read_csv('../data/station_attrs.csv')
embed = pd.read_csv('../data/station_embed.csv')
seq = pd.read_csv('../data/station_seq.csv')

### Select Sequence Values from Table

In [75]:
import pymysql
import pandas as pd
from private import db_info

host, user, password, db = db_info()

connection = pymysql.connect(
    host=host,
    user=user,
    password=password,
    db=db,
    charset='utf8',
    port = 3306
)

select_query = "SELECT * FROM sequence WHERE Time BETWEEN '2022-03-20 00:00:00' AND '2022-03-21 00:00:00'"

with connection.cursor() as cursor:
    cursor.execute(select_query)
    result = cursor.fetchall()

df = pd.DataFrame(result, columns=[col[0] for col in cursor.description])

In [85]:
# r, h, s, t, y로 만들기 위한 원상복구 코드.
df.pivot(index='Time', columns='Sid', values='Occupancy').reset_index().set_index('Time')

Sid,0,2,3,4,5,6,9,10,11,13,...,1046,1048,1049,1054,1055,1057,1058,1059,1060,1061
Time,,,,,,,,,,,,,,,,,,,,,
2022-03-20 00:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-03-20 00:20:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-03-20 00:40:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-03-20 01:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-03-20 01:20:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-20 22:40:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-03-20 23:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-03-20 23:20:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Update Output Values into Occupancy Table

In [ ]:
import pymysql
from private import db_info

host, user, password, db = db_info()

In [ ]:
def update_occupancy():

    connection = pymysql.connect(
    host=host,
    user=user,
    password=password,
    db=db,
    charset='utf8',
    port = 3306
)

    create_query = '''
CREATE TABLE occupancy (
Sid INT NOT NULL PRIMARY KEY, 
Occupancy_20 INT NOT NULL,
Occupancy_60 INT NOT NULL,
Occupancy_120 INT NOT NULL,
FOREIGN KEY (sid) REFERENCES station(sid)
)
'''
    select_query = "SELECT sid FROM station"

    try:
        with connection.cursor() as cursor:
                cursor.execute(create_query.replace('\n', ''))
                cursor.execute(select_query)
                result = cursor.fetchall()

        insert_query = "INSERT INTO occupancy VALUES {};"
        for idx in result:
            values = tuple(list(idx) + [0, 0, 0])
            with connection.cursor() as cursor:
                cursor.execute(insert_query.format(values))
            
        connection.commit()
        connection.close()

    except:
        with connection.cursor() as cursor:
            cursor.execute(select_query)
            result = cursor.fetchall()
        
        update_query = "UPDATE occupancy SET Occupancy_20 = %s, Occupancy_60 = %s, Occupancy_120 = %s WHERE Sid = %s"
        for idx in result:
            values = tuple([1, 1, 1] + list(idx))
            with connection.cursor() as cursor:
                cursor.execute(update_query, values)
            
        connection.commit()

In [ ]:
update_occupancy()